In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np

import torch
import torchvision as tv

import noiseprofile
import IntelSceneDataset as isds
import classifiers
from classifiers import SimpleClassifier

import denoise

from tqdm.auto import tqdm

from sporco import util
from sporco import linalg
from sporco import plot
plot.config_notebook_plotting()
from sporco.cupy import (cupy_enabled, np2cp, cp2np, select_device_by_load,
                         gpu_info)
from sporco.cupy.dictlrn import onlinecdl
from sporco.cupy.admm import cbpdn
from sporco.cupy import cnvrep
from sporco.cupy import linalg as cplinalg
from sporco.cupy import prox as cpprox
from sporco.cupy.linalg import irfftn,rfftn
from sporco.cupy.linalg import inner

import os

outpath = "./data/out"
inpath = "./data/in"
confpath = os.path.join(outpath, "conf")
noisepath = os.path.join(outpath, "noise")

import matplotlib.pyplot as plt

In [2]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
dbpath = "/mnt/hd-storage/IntelImageClassification"



cleanTe = isds.IntelSceneDataset(dbpath, segment="test", transform=transform)

In [4]:
# clf = SimpleClassifier(cleanTe.categories)
# clf.load("clean_sclf.pt")

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 34 * 34, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # relu preserves dimensions
        x = self.pool(torch.nn.functional.relu(self.conv1(x))) # in R^[N, Cout, |X|-|W|+1, |X|-|W|+1]
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, np.prod(x.shape[1:]))
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
clfname = "Resnet50CNN_NoPP"
    
clf = SimpleClassifier(cleanTe.categories, network=torch.load(os.path.join(inpath, f"{clfname}.pt")))

In [5]:
# noises = {
# #     "dropred": noise.ChannelDropoutNoise(0),
# #     "dropgreen": noise.ChannelDropoutNoise(1),
# #     "dropblue": noise.ChannelDropoutNoise(2),
#     "awn0": noise.AdditiveWhiteNoise(0, 0.1),
#     "srn0": noise.ScalarRayleighNoise(0.8),
#     "drop0": noise.DropoutNoise(0.05, axis=1),
#     "awn1": noise.AdditiveWhiteNoise(0, 0.2),
#     "srn1": noise.ScalarRayleighNoise(0.9),
#     "drop1": noise.DropoutNoise(0.1, axis=1),
#     "awn2": noise.AdditiveWhiteNoise(0, 0.3),
#     "srn2": noise.ScalarRayleighNoise(1.0),
#     "drop2": noise.DropoutNoise(0.15, axis=1),
#     "awn3": noise.AdditiveWhiteNoise(0, 0.4),
#     "srn3": noise.ScalarRayleighNoise(1.1),
#     "drop3": noise.DropoutNoise(0.2, axis=1),
#     "awn4": noise.AdditiveWhiteNoise(0, 0.5),
#     "srn4": noise.ScalarRayleighNoise(1.2),
#     "drop4": noise.DropoutNoise(0.25, axis=1),
#     "awn5": noise.AdditiveWhiteNoise(0, 0.6),
#     "srn5": noise.ScalarRayleighNoise(1.3),
#     "drop5": noise.DropoutNoise(0.3, axis=1)
# }

# noisepath = os.path.join(outpath, "noise")
# noisefnames = {k: os.path.join(noisepath, f"{k}_error.npz") for k in noises.keys()}


In [6]:
import noiseprofile
std = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
scale = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
rate = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
noiseparams = {
    "awn": [{"mean": 0, "std": s} for s in std],
    "srn": [{"scale": s} for s in scale],
    "drop": [{"rate": r} for r in rate]
}
print(noiseparams)
noises = {k: [None]*len(params) for k,params in noiseparams.items()}

for i in tqdm(range(len(noiseparams["awn"]))):
    fname = os.path.join(noisepath, f"Noise_awn{i}.npz")
    if os.path.exists(fname):
        print("Loading...")
        noises["awn"][i] = noiseprofile.AdditiveWhiteNoise.load(fname)
    else:
        print("Generating...")
        noises["awn"][i] = noiseprofile.AdditiveWhiteNoise(noiseparams["awn"][i])
    
for i in tqdm(range(len(noiseparams["srn"]))):
    fname = os.path.join(noisepath, f"Noise_srn{i}.npz")
    if os.path.exists(fname):
        print("Loading...")
        noises["srn"][i] = noiseprofile.ScalarRayleighNoise.load(fname)
    else:
        print("Generating...")
        noises["srn"][i] = noiseprofile.ScalarRayleighNoise(noiseparams["srn"][i])
    
for i in tqdm(range(len(noiseparams["drop"]))):
    fname = os.path.join(noisepath, f"Noise_drop{i}.npz")
    if os.path.exists(fname):
        print("Loading...")
        noises["drop"][i] = noiseprofile.DropoutNoise.load(fname)
    else:
        print("Generating...")
        noises["drop"][i] = noiseprofile.DropoutNoise(noiseparams["drop"][i])
    
print(noises)

{'awn': [{'mean': 0, 'std': 0.1}, {'mean': 0, 'std': 0.2}, {'mean': 0, 'std': 0.3}, {'mean': 0, 'std': 0.4}, {'mean': 0, 'std': 0.5}, {'mean': 0, 'std': 0.6}], 'srn': [{'scale': 0.7}, {'scale': 0.8}, {'scale': 0.9}, {'scale': 1.0}, {'scale': 1.1}, {'scale': 1.2}, {'scale': 1.3}], 'drop': [{'rate': 0.05}, {'rate': 0.1}, {'rate': 0.15}, {'rate': 0.2}, {'rate': 0.25}, {'rate': 0.3}]}


Loading...
Loading...
Loading...
Loading...
Loading...
Loading...



Loading...
Loading...
Loading...
Loading...
Loading...
Loading...
Loading...



Loading...
Loading...
Loading...
Loading...
Loading...
Loading...

{'awn': [<noiseprofile.AdditiveWhiteNoise object at 0x7fa95b63b090>, <noiseprofile.AdditiveWhiteNoise object at 0x7fa8920e9f90>, <noiseprofile.AdditiveWhiteNoise object at 0x7fa8920d6a10>, <noiseprofile.AdditiveWhiteNoise object at 0x7fa8920f2710>, <noiseprofile.AdditiveWhiteNoise object at 0x7fa8920d6e10>, <noiseprofile.AdditiveWhiteNoise object at 0x7fa89207c750>], 'srn': [<noiseprofile.ScalarRayleighNoise object at 0x7fa8920dd5d0>, <noiseprofile.ScalarRayleighNoise object at 0x7fa89208c090>, <noiseprofile.ScalarRayleighNoise object at 0x7fa8920dd310>, <noiseprofile.ScalarRayleighNoise object at 0x7fa892082090>, <noiseprofile.ScalarRayleighNoise object at 0x7fa89208cb90>, <noiseprofile.ScalarRayleighNoise object at 0x7fa892098dd0>, <noiseprofile.ScalarRayleighNoise object at 0x7fa8920ddf50>], 'drop': [<noiseprofile.DropoutNoise object at 0x7fa89203c150>, <noiseprofile.DropoutNoise object at 0x7fa89203c950>, <noiseprof

In [7]:
# for j in noises.keys():
#     for m in range(len(noises[j])):
#         noise = noises[j][m]
#         print(noises[j][m].param)
#         fname = os.path.join(noisepath, f"{j}{m}_error")
#         noise.save(fname)

In [8]:
# A = noiseprofile.DropoutNoise.load("./data/out/noise/drop2_error.npz")
# B = noises["drop"][2]
# print(A.param)
# print(B.param)
# print(type(A.epsilon))
# print(type(B.epsilon))
# print(B.size)
# print(A)
# print(B)

In [9]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batchsize = 8

In [10]:
dsTe = {
    k: [
        isds.IntelNoiseSceneDataset(cleanTe, delta)
            for delta in tqdm(noises[k], leave=False)
    ]
        for k in tqdm(noises.keys())
}
dsTe["clean"] = [cleanTe]


In [11]:
# Make sure noise is saved
for ntype in tqdm(noises.keys()):
    for nidx in tqdm(range(len(noises[ntype]))):
        fname = os.path.join(noisepath, f"Noise_{ntype}{nidx}.npz")
        if not os.path.exists(fname):
            noises[ntype][nidx].save(fname)

In [12]:
# # Sanity check that it was saved

# for i in range(len(noises["awn"])):
#     fname= os.path.join(noisepath, f"Noise_awn{i}.npz")
#     A = noiseprofile.AdditiveWhiteNoise.load(fname)
#     print(np.linalg.norm((A.epsilon - noises["awn"][i].epsilon).ravel()))

In [13]:
snrFlag = {k: True for k in dsTe.keys()}
snrFlag["clean"] = False

In [14]:
batchdims = denoise.CSC.batchdims(cleanTe.shape[2:], cleanTe.shape[1], batchsize)
batchkeys = {"sigshape", "channels", "batchsize"}
dictfname = "test_dict.npy"
csc = denoise.CSC(dictfname, batchdims, dimN=2, dtype=torch.float32)
print(csc.lmbda)

{'C': 3,
 'Cd': 3,
 'K': 8,
 'M': 240,
 'N': 22500,
 'Nv': (150, 150),
 'axisC': 2,
 'axisK': 3,
 'axisM': 4,
 'axisN': (0, 1),
 'dimC': 1,
 'dimCd': 1,
 'dimK': 1,
 'dimN': 2,
 'shpD': (10, 10, 3, 1, 240),
 'shpS': (150, 150, 3, 8, 1),
 'shpX': (150, 150, 1, 8, 240)}
0.05


In [15]:
# import denoise
# dataloader = torch.utils.data.DataLoader(dsTe["srn"],
#                                              batch_size=batchsize,
#                                              shuffle=True,
#                                              pin_memory=True,
#                                              num_workers=0)
# # print(dsTe[:10,...])
# noise_key = "srn"
# img_clean = dsTe["clean"][0].__getitem__(0)["images"].to(device)
# img_noise = dsTe[noise_key][2].__getitem__(0)["images"].to(device)
# img_recon = torch.clamp(csc.solve(img_noise.reshape((1,) + img_noise.shape), normed=True, lmbda=1.0e-2)[0,...], -1, 1).squeeze()
# print(img_recon.shape)

# # noise_diff = torch.abs(img_clean - img_noise)
# # recon_diff = torch.abs(img_clean - img_recon)

# # print(f"(min,max) = ({torch.min(img_clean):.3f}, {torch.max(img_clean):.3f})")
# # print(f"(min,max) = ({torch.min(img_noise):.3f}, {torch.max(img_noise):.3f})")
# # print(f"(min,max) = ({torch.min(img_recon):.3f}, {torch.max(img_recon):.3f})")

# # imgs = iter(dataloader).next().to(device)["images"]
# # rimgs = torch.clamp(csc.normsolve(imgs), -1, 1,)
# # dimgs = torch.abs(imgs - rimgs)
# # dimgs /= torch.max(dimgs)
# # print(f"(min,max) = ({torch.min(imgs):.3f}, {torch.max(imgs):.3f})")
# # print(f"(min,max) = ({torch.min(rimgs):.3f}, {torch.max(rimgs):.3f})")
# # print(f"(min,max) = ({torch.min(dimgs):.3f}, {torch.max(dimgs):.3f})")
# # for m in range(imgs.shape[0]):
# #     plt.subplot(2,2,1)
# #     plt.imshow(0.5*imgs[m,...].permute((1,2,0)).cpu() + 0.5)
# #     plt.subplot(2,2,2)
# #     plt.imshow(dimgs[m,...].permute((1,2,0)).cpu())
# #     plt.subplot(2,2,3)
# #     plt.imshow(0.5*rimgs[m,...].permute((1,2,0)).cpu() + 0.5)
# #     plt.subplot(2,2,4)
# #     plt.imshow(0.5*torch.abs(rimgs[m,...].permute((1,2,0)).cpu() + 0.5))
# #     plt.show()

In [16]:
# cpuperm = lambda x: x.permute(1,2,0).cpu()
# norm = lambda x: 0.5*cpuperm(x) + 0.5
# diffnorm = lambda x: cpuperm(x.div_(torch.max(x)))
# fig = plt.subplots(2,2, figsize=(10,10))
# plt.subplot(2,2,1)
# plt.imshow(norm(img_noise))
# plt.subplot(2,2,2)
# plt.imshow(diffnorm(noise_diff))
# plt.subplot(2,2,3)
# plt.imshow(norm(img_recon))
# plt.subplot(2,2,4)
# plt.imshow(diffnorm(recon_diff))

In [17]:
# csc.lmbda = 1.0e-2 # 5.0e-2
# for k, data in dsTe.items():
#     dataloader = torch.utils.data.DataLoader(data,
#                                              batch_size=batchsize,
#                                              shuffle=True,
#                                              pin_memory=True,
#                                              num_workers=0)
#     if snrFlag[k]:
#         Cnoise[k], SNRnoise[k] = clf.predict(dataloader, calcsnr=snrFlag[k], window=1)
#         print("No preprocessing:")
#         print("%s Accuracy: %.4f" % (k, np.sum(np.diag(Cnoise[k])) / np.sum(Cnoise[k])))
#         print("%s Mean SNR: %f dB" % (k, np.mean(SNRnoise[k])))
    
#         Ccdl[k], SNRcdl[k] = clf.predict(dataloader, calcsnr=snrFlag[k], window=1, preprocessor=csc.normsolve)
#         print("Preprocessing with CDL:")
#         print("%s Accuracy: %.4f" % (k, np.sum(np.diag(Ccdl[k])) / np.sum(Ccdl[k])))
#         print("%s Mean SNR: %f dB" % (k, np.mean(SNRcdl[k])))
#         print("============================================================================")
#     else:
#         Cnoise[k], _ = clf.predict(dataloader, calcsnr=snrFlag[k], window=1)
#         print("No preprocessing:")
#         print("%s Accuracy: %.4f" % (k, np.sum(np.diag(Cnoise[k])) / np.sum(Cnoise[k])))
    
#         Ccdl[k], _ = clf.predict(dataloader, calcsnr=snrFlag[k], window=1, preprocessor=csc.normsolve)
#         print("Preprocessing with CDL:")
#         print("%s Accuracy: %.4f" % (k, np.sum(np.diag(Ccdl[k])) / np.sum(Ccdl[k])))
#         print("============================================================================")

In [ ]:
import IntelSceneDataset
import classifiers

confopath = os.path.join(outpath, "conf")

csc.lmbda = 1.0e-2 # 5.0e-2
numcats = len(dsTe["clean"][0].categories)
numsamps = len(dsTe["clean"][0])
A = np.zeros((numcats, numcats))
Conf = {
    "Baseline": {k: [np.zeros((numcats, numcats))]*len(dsTe[k]) for k in dsTe.keys()},
    "CDL": {k: [np.zeros((numcats, numcats))]*len(dsTe[k]) for k in dsTe.keys()}
}
SNR = {
    "Baseline": {k: [np.zeros(numsamps)]*len(dsTe[k]) for k in dsTe.keys()},
    "CDL": {k: [np.zeros(numsamps)]*len(dsTe[k]) for k in dsTe.keys()}
}
Pred = {
    "Baseline": {k: [np.zeros(numsamps)]*len(dsTe[k]) for k in dsTe.keys()},
    "CDL": {k: [np.zeros(numsamps)]*len(dsTe[k]) for k in dsTe.keys()}
}
Preprocessor = {
    "Baseline": None,
    "CDL": csc.normsolve
}
for nkey,datalist in dsTe.items():
    for nidx in range(len(datalist)):
        for pkey,pp in Preprocessor.items():
            confname = os.path.join(confopath, f"{clfname}_Confussion_{pkey}_{nkey}{nidx}.npz")
            errstr = "None" if nkey == "clean" else str(noises[nkey][nidx])
            if os.path.exists(confname):
                print(f"Skipping {pkey}: {errstr}")
                continue
            dataloader = torch.utils.data.DataLoader(datalist[nidx],
                                                batch_size=batchsize,
                                                shuffle=True,
                                                pin_memory=True,
                                                num_workers=0)
            preddct = clf.predict(dataloader, calcsnr=snrFlag[ntype], window=1, preprocessor=pp)
            Conf[pkey][nkey][nidx] = preddct["Conf"].copy()
            Pred[pkey][nkey][nidx] = preddct["labels"].copy()
            print(f"{nkey} {nidx} Average Accuracy: {np.sum(np.diag(preddct['Conf'])) / np.sum(preddct['Conf']):.4f}")
            if snrFlag[nkey]:
                SNR[pkey][nkey][nidx] = preddct["SNR"].copy()
                print(f"{nkey} {nidx} Median SNR: {np.median(SNR[pkey][nkey][nidx]):.4f} dB")
            print("\n")
            
            try:
                np.savez_compressed(confname,
                                    C=Conf[pkey][nkey][nidx],
                                    SNR=SNR[pkey][nkey][nidx],
                                    Pred=Pred[pkey][nkey][nidx])
            except Exception as e:
                print(e.message, e.args)
        print("============================================================================")
            
            
            
# for k, data in dsTe.items():
#     dataloader = torch.utils.data.DataLoader(data,
#                                              batch_size=batchsize,
#                                              shuffle=True,
#                                              pin_memory=True,
#                                              num_workers=0)
#     if k == "clean":
#         break
#     for j in Conf.keys():
#         preddct = clf.predict(dataloader, calcsnr=snrFlag[k], window=1, preprocessor=Preprocessor[j])
#         Conf[j][k] = preddct["Conf"]
#         Pred[j][k] = preddct["labels"]
#         print(f"{j}:")
#         print(f"{k} Accuracy: {np.sum(np.diag(Conf[j][k])) / np.sum(Conf[j][k]):.4f}")
#         if snrFlag[k]:
#             SNR[j][k] = preddct["SNR"]
#             print(f"{k} Mean SNR: {np.mean(SNR[j][k]):.4f} dB")
#         print("\n")
#         confname = os.path.join(confopath, f"{clfname}_Confussion_{j}_{k}.npz")
#         try:
#             np.savez_compressed(confname, C=Conf[j][k], SNR=SNR[j][k], Pred=Pred[j][k])
#         except Exception as e:
#             print(e.message, e.args)
#     print("============================================================================")

Skipping Baseline: Normal$\left(0, 0.1\right)$
Skipping CDL: Normal$\left(0, 0.1\right)$
Skipping Baseline: Normal$\left(0, 0.2\right)$
Skipping CDL: Normal$\left(0, 0.2\right)$
Skipping Baseline: Normal$\left(0, 0.3\right)$
Skipping CDL: Normal$\left(0, 0.3\right)$
Skipping Baseline: Normal$\left(0, 0.4\right)$
Skipping CDL: Normal$\left(0, 0.4\right)$
Skipping Baseline: Normal$\left(0, 0.5\right)$


awn 4 Average Accuracy: 0.5067
awn 4 Median SNR: 1.7913 dB




awn 5 Average Accuracy: 0.4290
awn 5 Median SNR: 1.4235 dB




awn 5 Average Accuracy: 0.4567
awn 5 Median SNR: 1.4235 dB




srn 0 Average Accuracy: 0.6680
srn 0 Median SNR: 3.7555 dB




In [ ]:
A = np.load(f"./data/out/conf/{clfname}_Confussion_Baseline_srn6.npz")
print(A['Pred'].shape)
print(A['C'].shape)
print(A['SNR'].shape)
print(dsTe["clean"][0].y.shape)
# for k,C in Ccdl.items():
#     fname = str(k) + "_" + confcdlfname
#     try:
#         np.savez_compressed(fname, k=C)
#     except Exception as e:
#         print(e.message, e.args)

# confnoisefname = "ConfusionNoise.npz"

# for k,C in Cnoise.items():
#     fname = str(k) + "_" + confnoisefname
#     try:
#         np.savez_compressed(fname, k=C)
#     except Exception as e:
#         print(e.message, e.args)